<a href="https://colab.research.google.com/github/PabloGuev17/DatosGeoEspaciales/blob/main/MCD_CDGE_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías de Python para Ciencia de Datos Geoespaciales.


## 1. Librerías de Python

Durante el desarrollo del curso, utilizaremos (al menos) las siguientes librerías de Python:

- geopandas
- contextily
- earthengine-api
- earthpy
- fiona
- folium
- geemap
- geotiff
- leafmap
- mapclassify
- netcdf
- osmnx
- pysal
- rasterio
- rasterstats
- rioxarray
- s2cloudless
- shapely
- xarray
- xarray-spatial
- xee

Estas librerías pueden ser instaladas en forma independiente, usando comandos como `pip install` o `conda install`: Por ejemplo, puedes ejecutar:




También es posible instalar directamente el paquete `geospatial`, que agrupa las librerías típicamente utilizadas para análisis geoespacial:

https://geospatial.gishub.org

- Vea la documentación de `geospatial` y siga las instrucciones de instalación.
- En caso de fallas en el proceso de instalación, se sugiere copiar el mensaje de error, y hacer una búsqueda en Google o en foros como https://gis.stackexchange.com o https://stackoverflow.com/questions. Es muy probable que alguien haya enfrentado (y resuelto) antes el mismo problema.

- Si el problema persiste, por favor copiar el mensaje de error y enviarlo por correo a la profesora y ayudante.



- Para probar su instalación, puede ejecutar por ejemplo los siguientes comandos:

In [ ]:
import pandas as pd
import geopandas as gpd
import ee,geemap
import rioxarray as rxr
import contextily as ctx
import xarray as xr
import folium
import pysal

In [ ]:
import leafmap
m = leafmap.Map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## 2. Cuenta personal en Google Earth Engine.

Más adelante utilizaremos la plataforma Google Earth Engine, que requiere el registro y creación de una cuenta personal gratuita. Para crear su cuenta, sigua las instrucciones en el siguiente link:

https://code.earthengine.google.com/register

Seleccione las siguientes opciones:
- **Unpaid usage**
- Project type: **Academia & Research**
- **Create a new Google Cloud Project** (ingrese un nombre a su elección en Project-ID)
- Desde Septiembre 2025, Google incluye en el flujo de creación de cuenta, la obligación de registrar un proyecto, y validar que éste cumple con las condiciones para uso no-comercial. Este registro toma un par de minutos y sólo es nevesrio hacerlo una vez.

Para probar el uso de tu cuenta, ejectua la siguiente celda. El comando *ee.Authenticate()* genera un flujo de aprobaciones sucesivas.



In [ ]:
import ee
ee.Authenticate()

True

In [ ]:
ee.Initialize(project='ee-aguirrepaula')

Cuando ejecutes la siguiente celda, deberías ver una imagen satelital superpuesta sobre la capda de fondo de Open Street Map.

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

# Pre-filter to get less cloudy granules#
dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') .filterDate('2020-01-01', '2020-01-30')
     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(83.277, 17.7009, 12)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…